# Autonomous NALU System
This system aides in real time data gathering for Autonomous training and even shortest path/routes by remmbering and counting stops and any other delay durations to maximize effeciant Auotnomous Driving and Flying.

In [1]:
from __future__  import print_function
import cv2
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable


### Live Cam data feed

In [ ]:
def start_tracking():
 
    cap = cv2.VideoCapture(0)
    scaling_factor = 0.5
    num_frames_to_track = 5
    num_frames_jump = 2
    tracking_paths = []
    frame_index = 0
    
    tracking_params = dict(winSize  = (28, 28), maxLevel = 2,
            criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                10, 0.03))

    while True:
        _, frame = cap.read()
        frame = cv2.resize(frame, None, fx=scaling_factor, 
                fy=scaling_factor, interpolation=cv2.INTER_AREA)

     
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

       
        output_img = frame.copy()

        if len(tracking_paths) > 0:
            prev_img, current_img = prev_gray, frame_gray
            feature_points_0 = np.float32([tp[-1] for tp in tracking_paths]).reshape(-1, 1, 2)
            
            feature_points_1, _, _ = cv2.calcOpticalFlowPyrLK(
                    prev_img, current_img, feature_points_0, 
                    None, **tracking_params)

            feature_points_0_rev, _, _ = cv2.calcOpticalFlowPyrLK(
                    current_img, prev_img, feature_points_1, 
                    None, **tracking_params)

            diff_feature_points = abs(feature_points_0 - feature_points_0_rev).reshape(-1, 2).max(-1)

            good_points = diff_feature_points < 1
            new_tracking_paths = []

            for tp, (x, y), good_points_flag in zip(tracking_paths, 
                        feature_points_1.reshape(-1, 2), good_points):
    
                if not good_points_flag:
                    continue

                tp.append((x, y))
                if len(tp) > num_frames_to_track:
                    del tp[0]

                new_tracking_paths.append(tp)
                cv2.circle(output_img, (x, y), 3, (0, 255, 0), -1)

    
            tracking_paths = new_tracking_paths

            cv2.polylines(output_img, [np.int32(tp) for tp in tracking_paths], False, (0, 150, 0))

        if not frame_index % num_frames_jump:
            mask = np.zeros_like(frame_gray)
            mask[:] = 255
            for x, y in [np.int32(tp[-1]) for tp in tracking_paths]:
                cv2.circle(mask, (x, y), 6, 0, -1)
                
            feature_points = cv2.goodFeaturesToTrack(frame_gray, 
                    mask = mask, maxCorners = 500, qualityLevel = 0.3, 
                    minDistance = 7, blockSize = 7) 

            if feature_points is not None:
                for x, y in np.float32(feature_points).reshape(-1, 2):
                    tracking_paths.append([(x, y)])

        frame_index += 1
        prev_gray = frame_gray
        
    
        c = cv2.waitKey(1)
        if c == 27:
            break


if __name__ == '__main__':

    start_tracking()
    cv2.destroyAllWindows()

### NALU

In [2]:
def nalu(input_layer, num_outputs):

    shape = (int(input_layer.shape[-1]), num_outputs)

    W_hat = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
    M_hat = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
    G = tf.Variable(tf.truncated_normal(shape, stddev=0.02))

    W = tf.tanh(W_hat) * tf.sigmoid(M_hat)
    m = tf.exp(tf.matmul(tf.log(tf.abs(input_layer) + 1e-7), W))
    g = tf.sigmoid(tf.matmul(input_layer, G))
    a = tf.matmul(input_layer, W)
    out = g * a + (1 - g) * m

    return out


### Training

In [4]:
EPOCS = 5
LEARNING_RATE = 0.9
BATCH_SIZE = 10

  
X_data, Y_data = output_img(op='prod')
X = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 2])
Y_true = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 1])
Y_pred = nalu(X, 1)
loss = tf.nn.l2_loss(Y_pred - Y_true) 
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train_op = optimizer.minimize(loss)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for ep in range(EPOCS):
    i = 0
    gts = 0
        
    while i < len(X_data):
        xs, ys = X_data[i:i+BATCH_SIZE], Y_data[i:i+BATCH_SIZE]

        _, ys_pred, l = sess.run([train_op, Y_pred, loss], 
                feed_dict={X: xs, Y_true: ys})

        gts += np.sum(np.isclose(ys, ys_pred, atol=1e-4, rtol=1e-4)) 

        i += BATCH_SIZE

        acc = gts/len(Y_data)
        print('epoch {2}, loss: {0}, accuracy: {1}'.format(l, acc, ep))

epoch 0, loss: 1714.901611328125, accuracy: 0.0
epoch 0, loss: 1858.658935546875, accuracy: 0.0
epoch 0, loss: 952.3518676757812, accuracy: 0.0001
epoch 0, loss: 1209.190185546875, accuracy: 0.0002
epoch 0, loss: 373.5921936035156, accuracy: 0.0002
epoch 0, loss: 340.3475646972656, accuracy: 0.0004
epoch 0, loss: 21.16185188293457, accuracy: 0.0006
epoch 0, loss: 28.34718132019043, accuracy: 0.0006
epoch 0, loss: 4.014919281005859, accuracy: 0.0008
epoch 0, loss: 0.5873929858207703, accuracy: 0.0011
epoch 0, loss: 0.3078399896621704, accuracy: 0.0013
epoch 0, loss: 0.2898422181606293, accuracy: 0.0013
epoch 0, loss: 0.06032023951411247, accuracy: 0.0017
epoch 0, loss: 0.017906276509165764, accuracy: 0.0019
epoch 0, loss: 0.008699623867869377, accuracy: 0.0022
epoch 0, loss: 0.003612136235460639, accuracy: 0.0027
epoch 0, loss: 0.0023542640265077353, accuracy: 0.0029
epoch 0, loss: 0.0035769047681242228, accuracy: 0.0029
epoch 0, loss: 0.001533386530354619, accuracy: 0.003
epoch 0, loss

epoch 0, loss: 2.0304838471929543e-05, accuracy: 0.2538
epoch 0, loss: 9.294130904891063e-06, accuracy: 0.2548
epoch 0, loss: 3.190055213053711e-05, accuracy: 0.2558
epoch 0, loss: 1.3695874258701224e-05, accuracy: 0.2568
epoch 0, loss: 1.3572830539487768e-05, accuracy: 0.2578
epoch 0, loss: 1.1633739632088691e-05, accuracy: 0.2588
epoch 0, loss: 1.0875930456677452e-05, accuracy: 0.2598
epoch 0, loss: 3.8267040508799255e-05, accuracy: 0.2608
epoch 0, loss: 6.810003014834365e-06, accuracy: 0.2618
epoch 0, loss: 1.573967165313661e-05, accuracy: 0.2628
epoch 0, loss: 2.9106569854775444e-05, accuracy: 0.2638
epoch 0, loss: 3.390271376702003e-05, accuracy: 0.2648
epoch 0, loss: 3.3213538699783385e-05, accuracy: 0.2658
epoch 0, loss: 4.6782060962868854e-05, accuracy: 0.2667
epoch 0, loss: 4.309617361286655e-05, accuracy: 0.2676
epoch 0, loss: 1.938170134963002e-05, accuracy: 0.2686
epoch 0, loss: 7.203893801488448e-06, accuracy: 0.2696
epoch 0, loss: 2.135149043169804e-05, accuracy: 0.2704
e

epoch 0, loss: 1.0090418072650209e-05, accuracy: 0.484
epoch 0, loss: 1.0497541552467737e-05, accuracy: 0.485
epoch 0, loss: 6.656090590695385e-06, accuracy: 0.486
epoch 0, loss: 2.2547970729647204e-05, accuracy: 0.487
epoch 0, loss: 6.638531885982957e-06, accuracy: 0.488
epoch 0, loss: 2.157853487005923e-05, accuracy: 0.4889
epoch 0, loss: 1.504096326243598e-05, accuracy: 0.4899
epoch 0, loss: 3.25931869156193e-05, accuracy: 0.4908
epoch 0, loss: 2.3645698092877865e-05, accuracy: 0.4918
epoch 0, loss: 2.8208967705722898e-05, accuracy: 0.4927
epoch 0, loss: 4.819329660676885e-06, accuracy: 0.4937
epoch 0, loss: 1.3158049114281312e-05, accuracy: 0.4946
epoch 0, loss: 2.3312608391279355e-05, accuracy: 0.4956
epoch 0, loss: 1.1406285921111703e-05, accuracy: 0.4966
epoch 0, loss: 6.739402124367189e-06, accuracy: 0.4976
epoch 0, loss: 3.5262128221802413e-05, accuracy: 0.4985
epoch 0, loss: 1.9634624550235458e-05, accuracy: 0.4995
epoch 0, loss: 9.997307643061504e-06, accuracy: 0.5005
epoch 

epoch 0, loss: 8.377615813515149e-06, accuracy: 0.6943
epoch 0, loss: 2.302494249306619e-05, accuracy: 0.6953
epoch 0, loss: 2.8294116418692283e-06, accuracy: 0.6963
epoch 0, loss: 2.3027909264783375e-05, accuracy: 0.6972
epoch 0, loss: 2.081877028103918e-05, accuracy: 0.6981
epoch 0, loss: 1.8152431948692538e-05, accuracy: 0.699
epoch 0, loss: 2.0791052520507947e-05, accuracy: 0.6999
epoch 0, loss: 5.2638002671301365e-05, accuracy: 0.7008
epoch 0, loss: 2.172194217564538e-05, accuracy: 0.7018
epoch 0, loss: 5.9392630646470934e-05, accuracy: 0.7027
epoch 0, loss: 1.5476311091333628e-05, accuracy: 0.7036
epoch 0, loss: 1.4139205632091034e-05, accuracy: 0.7046
epoch 0, loss: 2.8872086659248453e-06, accuracy: 0.7056
epoch 0, loss: 1.0571703569439705e-05, accuracy: 0.7066
epoch 0, loss: 3.683703835122287e-05, accuracy: 0.7076
epoch 0, loss: 2.9661416192539036e-05, accuracy: 0.7085
epoch 0, loss: 3.463912435108796e-05, accuracy: 0.7094
epoch 0, loss: 1.980072192964144e-05, accuracy: 0.7104


epoch 0, loss: 2.301702079421375e-05, accuracy: 0.9168
epoch 0, loss: 4.367799192550592e-05, accuracy: 0.9177
epoch 0, loss: 1.531582347524818e-05, accuracy: 0.9187
epoch 0, loss: 1.0513145753066055e-05, accuracy: 0.9197
epoch 0, loss: 2.0458011931623332e-05, accuracy: 0.9207
epoch 0, loss: 1.3514278180082329e-05, accuracy: 0.9217
epoch 0, loss: 1.0882625247177202e-05, accuracy: 0.9227
epoch 0, loss: 6.387738721969072e-06, accuracy: 0.9237
epoch 0, loss: 3.398756598471664e-05, accuracy: 0.9246
epoch 0, loss: 2.1335041310521774e-05, accuracy: 0.9256
epoch 0, loss: 1.6441274055978283e-05, accuracy: 0.9266
epoch 0, loss: 7.6281467045191675e-06, accuracy: 0.9276
epoch 0, loss: 4.107453059987165e-05, accuracy: 0.9286
epoch 0, loss: 3.114751598332077e-05, accuracy: 0.9295
epoch 0, loss: 2.2228186935535632e-05, accuracy: 0.9305
epoch 0, loss: 1.2006601536995731e-05, accuracy: 0.9314
epoch 0, loss: 1.0799270057759713e-05, accuracy: 0.9324
epoch 0, loss: 1.596478614374064e-05, accuracy: 0.9334


...

epoch 4, loss: 1.2811176929972135e-05, accuracy: 0.1123
epoch 4, loss: 1.4088691386859864e-05, accuracy: 0.1133
epoch 4, loss: 2.654180752870161e-05, accuracy: 0.1143
epoch 4, loss: 3.191802170476876e-05, accuracy: 0.1153
epoch 4, loss: 2.5163750251522288e-05, accuracy: 0.1163
epoch 4, loss: 1.217657973029418e-05, accuracy: 0.1172
epoch 4, loss: 1.514831183158094e-05, accuracy: 0.1182
epoch 4, loss: 1.7828389900387265e-05, accuracy: 0.1192
epoch 4, loss: 3.980137262260541e-05, accuracy: 0.1201
epoch 4, loss: 5.63105686524068e-06, accuracy: 0.1211
epoch 4, loss: 1.813191920518875e-05, accuracy: 0.1221
epoch 4, loss: 1.5601090126438066e-05, accuracy: 0.1231
epoch 4, loss: 2.510991907911375e-05, accuracy: 0.124
epoch 4, loss: 3.170365380356088e-05, accuracy: 0.125
epoch 4, loss: 1.7240770830539986e-05, accuracy: 0.126
epoch 4, loss: 3.289684173068963e-05, accuracy: 0.127
epoch 4, loss: 3.180325074936263e-05, accuracy: 0.128
epoch 4, loss: 1.4222396202967502e-05, accuracy: 0.129
epoch 4, l

epoch 4, loss: 1.926378536154516e-05, accuracy: 0.3944
epoch 4, loss: 1.3699031114811078e-05, accuracy: 0.3954
epoch 4, loss: 2.6114097636309452e-05, accuracy: 0.3964
epoch 4, loss: 6.134428076620679e-06, accuracy: 0.3974
epoch 4, loss: 2.9667653507203795e-05, accuracy: 0.3984
epoch 4, loss: 2.7824760763905942e-05, accuracy: 0.3994
epoch 4, loss: 1.4588820704375394e-05, accuracy: 0.4004
epoch 4, loss: 1.3363434845814481e-05, accuracy: 0.4014
epoch 4, loss: 2.957102333311923e-05, accuracy: 0.4024
epoch 4, loss: 5.531667557079345e-05, accuracy: 0.4034
epoch 4, loss: 1.2538928785943426e-05, accuracy: 0.4044
epoch 4, loss: 2.935567681561224e-05, accuracy: 0.4054
epoch 4, loss: 2.1511139493668452e-05, accuracy: 0.4064
epoch 4, loss: 1.788825829862617e-05, accuracy: 0.4074
epoch 4, loss: 4.66054098069435e-06, accuracy: 0.4084
epoch 4, loss: 2.6555628210189752e-05, accuracy: 0.4094
epoch 4, loss: 9.399956979905255e-06, accuracy: 0.4104
epoch 4, loss: 3.425419345148839e-05, accuracy: 0.4114
ep

epoch 4, loss: 4.119776713196188e-05, accuracy: 0.6822
epoch 4, loss: 1.6060037523857318e-05, accuracy: 0.6832
epoch 4, loss: 1.7232920072274283e-05, accuracy: 0.6842
epoch 4, loss: 6.10853248872445e-06, accuracy: 0.6852
epoch 4, loss: 2.2598191208089702e-05, accuracy: 0.6862
epoch 4, loss: 2.6347879611421376e-05, accuracy: 0.6872
epoch 4, loss: 2.037560443568509e-05, accuracy: 0.6882
epoch 4, loss: 1.001633245323319e-05, accuracy: 0.6892
epoch 4, loss: 3.3097894629463553e-05, accuracy: 0.6901
epoch 4, loss: 1.3420576578937471e-05, accuracy: 0.6911
epoch 4, loss: 3.808719338849187e-05, accuracy: 0.6921
epoch 4, loss: 4.08357082051225e-05, accuracy: 0.6929
epoch 4, loss: 2.7877564207301475e-05, accuracy: 0.6939
epoch 4, loss: 1.5101060853339732e-05, accuracy: 0.6949
epoch 4, loss: 2.903027598222252e-05, accuracy: 0.6959
epoch 4, loss: 5.58811952942051e-05, accuracy: 0.6968
epoch 4, loss: 3.000472679559607e-05, accuracy: 0.6978
epoch 4, loss: 2.2090194761403836e-05, accuracy: 0.6988
epoc

epoch 4, loss: 1.4500200450129341e-05, accuracy: 0.8268
epoch 4, loss: 1.0178351658396423e-05, accuracy: 0.8278
epoch 4, loss: 2.3881304514361545e-05, accuracy: 0.8287
epoch 4, loss: 3.302103505120613e-05, accuracy: 0.8297
epoch 4, loss: 1.93693285837071e-05, accuracy: 0.8307
epoch 4, loss: 3.4108172258129343e-05, accuracy: 0.8317
epoch 4, loss: 3.9510421629529446e-05, accuracy: 0.8327
epoch 4, loss: 3.0711819363204995e-06, accuracy: 0.8337
epoch 4, loss: 1.4905122952768579e-05, accuracy: 0.8347
epoch 4, loss: 2.723261968640145e-05, accuracy: 0.8357
epoch 4, loss: 8.967919711722061e-06, accuracy: 0.8367
epoch 4, loss: 1.3107115591992624e-05, accuracy: 0.8377
epoch 4, loss: 2.6864856408792548e-05, accuracy: 0.8387
epoch 4, loss: 2.173093434976181e-06, accuracy: 0.8397
epoch 4, loss: 4.497093323152512e-05, accuracy: 0.8406
epoch 4, loss: 1.683562913967762e-05, accuracy: 0.8416
epoch 4, loss: 2.596704325696919e-05, accuracy: 0.8425
epoch 4, loss: 2.5952795112971216e-05, accuracy: 0.8435
e

epoch 4, loss: 2.1350257156882435e-05, accuracy: 0.9718
epoch 4, loss: 1.1780392014770769e-05, accuracy: 0.9727
epoch 4, loss: 1.040745701175183e-05, accuracy: 0.9737
epoch 4, loss: 1.5293233445845544e-05, accuracy: 0.9747
